# PFun CMA Model -- Modular Demos

In [ ]:
import pfun_cma_model as pfun

## Instantiate the model

In [ ]:
import json
cma = pfun.CMASleepWakeModel()
cma.to_dict()

## Run the model (24 time bins)

In [ ]:
cma.run()

## Load & Process an example dataset

In [ ]:
from pfun_cma_model.engine.fit import fit_model
from pfun_cma_model.engine.data_utils import format_data
from pfun_cma_model.misc.pathdefs import PFunDataPaths

df = PFunDataPaths().read_sample_data()

print('(raw) sample data')
print(df.tail().to_markdown())
print()

print('formatted sample data')
df_formatted = format_data(df)
print(df_formatted.tail().to_markdown())
print()

ax = df_formatted.plot(
    x='time',
    y=['G'],
    title='Sample Glucose Dataset',
    figsize=(10, 5),
    linestyle='',
    marker='o',
)

#### Zoom on Last day (raw data)

In [ ]:
ax.set_xlim('2021-01-27 16:00:00', df_formatted['time'].max())
ax.get_figure()

## Fit the model to sample data

### Raw fit result (solution)

In [ ]:
print('model fit result')
result = fit_model(
    df,
    tm_freq='1h',
    curve_fit_kwds={
        'ftol': 1e-18,
        'xtol': 1e-18,
        'method': 'L-BFGS-B',
        })
print(result.soln.shape)
print(result)
print()

### Raw fitted parameters

**Displays `{ param_name: type(param_value) }` for all fitted parameters in the model result.**

In [ ]:
import pandas as pd
raw_params = result.cma.params

print('types of raw parameters')
params_types = {k: ( str(type(v)), repr(v)[:100] + '...' ) for k, v in raw_params.items()}

df_params_types = pd.DataFrame.from_dict(
    params_types, orient='index', columns=['type', 'repr_head'])
df_params_types.sort_values(by='type', inplace=False)

## CMAModelParams: Pretty Markdown Table

**Includes qualitative descriptions (e.g., 'Normal', 'Low', 'High')**

In [ ]:
from pfun_cma_model import CMAModelParams

params = result.cma.bounded_params
print(params.generate_markdown_table())

### Fitted Solution Plotted

In [ ]:
result.soln.plot(
    x='t',
    y=['G'],
    title='Model Fit Result',
    figsize=(10, 5),
    linestyle='',
    marker='x',
    logy=True
)

#### Zoomed on dinner estimate (fitted)

In [ ]:
result.soln.iloc[-200:, :].plot(x='t', y=['G'], figsize=(10, 5), linestyle='-', marker='o')

In [ ]:
from pfun_cma_model.engine.cma_plot import CMAPlotConfig

cma_plot = CMAPlotConfig()

cma_plot.plot_model_results(
    result.formatted_data,
    result.soln,
    as_blob=False,
    separate2subplots=False,
)

In [ ]:
from pfun_cma_model.engine.cma_plot import CMAPlotConfig

cma_plot = CMAPlotConfig()

cma_plot.plot_model_results(
    result.formatted_data,
    result.soln,
    as_blob=False,
    separate2subplots=True,
)

## Experiments with N=...

In [ ]:
for n in [12, 24, 288, 1024]:
    print(f'Running model with {n} time points')
    soln = pfun.CMASleepWakeModel(N=n).run()
    print(soln.shape)
    print()

# CMA Model Parameters (Curve fitting)

Experiments
-----------

[./cma-model-tuning.ipynb](./cma-model-tuning.ipynb)